## Scraping all tables from http://www.planecrashinfo.com/database.htm
#### We will now construct a scrapper, to fetch all info on the tables and append into a dataframe

In [2]:
# importing all the necessay libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

### Create all the links

#### There's a link for each year, and inside that link, there's a table with multiple accidents <br> links for the years are constructed as planecrashinfo.com/year/year.htm <br> First I iterated through all years and stored these links into an empty list "links" <br>Then a not so elegant solution was implement to remove the substring ".html" <br>The links for the actual data have an -n per table/year <br> So a last for loop was used to append a -n from 1 to 150 to the links and store them in a new list

In [3]:
url = "http://www.planecrashinfo.com/database.htm"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

links = []
#assign the links scraped into the var links
for link in soup.find_all("a"):
    if not link.get("href").startswith("/"):
        links.append("http://www.planecrashinfo.com/" + link.get("href"))
    else:
        links.append("http://www.planecrashinfo.com" + link.get("href"))

#remoeve the last 4 strings from each url
links_sliced = [link[:-4] for link in links]

#append -1 to -150 to each url, do to the way the urls are constructed can't find a better solution
new_links = []
for link in links_sliced:
    for i in range(-1, -151, -1):
        new_link = link + str(i) + str('.htm')
        new_links.append(new_link)

#### Since all info is stored inside an html table a for loop was used to iterate through the columns I was interested in on the first loop iterating through all the links stored on the list created above <br> This data was then stored in a dataframe

In [8]:
#dataframe
crashes = pd.DataFrame(columns=["Date", "Time","Location","Operator", "Flight_Number", "Route", "AC_Type", "Registration", "cn_ln", "Aboard", "Fatalities", "Ground", "Summary"])

#look for the tables
for url in new_links:
    data = requests.get(url).text
    soup = BeautifulSoup(data,"html.parser")
    tables = soup.find_all('table')

    #indexing the tables
    for index,table in enumerate(tables):
        if ("ACCIDENT DETAILS" in str(table)):
            table_index = index

    #constructing the table
    rows = tables
    for row in rows:
        cols = row.find_all("td")
        if (cols != []):
            Date = cols[3].text
            Time = cols[5].text
            Location= cols[7].text
            Operator = cols[9].text
            Flight_Number = cols[11].text
            Route = cols[13].text
            AC_Type = cols[15].text
            Registration = cols[17].text
            cn_ln = cols[19].text
            Aboard = cols[21].text
            Fatalities = cols[23].text
            Ground = cols[25].text
            Summary = cols[27].text
            crashes = crashes.append({"Date":Date, "Time":Time, 'Location':Location, "Operator":Operator, "Flight_Number":Flight_Number, "Route":Route, "AC_Type":AC_Type, "Registration":Registration, "cn_ln":cn_ln, "Aboard":Aboard, "Fatalities":Fatalities, "Ground":Ground, "Summary":Summary}, ignore_index=True)

crashes

,Date,Time,Location,Operator,Flight_Number,Route,AC_Type,Registration,cn_ln,Aboard,Fatalities,Ground,Summary
0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,2 (passengers:1 crew:1),1 (passengers:1 crew:0),0,"During a demonstration flight, a U.S. Army fly..."
1,"September 07, 1909",?,"Juvisy-sur-Orge, France",?,?,Air show,Wright Byplane,SC1,?,1 (passengers:0 crew:1),1 (passengers:0 crew:0),0,Eugene Lefebvre was the first pilot to ever be...
2,"July 12, 1912",0630,"Atlantic City, New Jersey",Military - U.S. Navy,?,Test flight,Dirigible,?,?,5 (passengers:0 crew:5),5 (passengers:0 crew:5),0,First U.S. dirigible Akron exploded just offsh...
3,"August 06, 1913",?,"Victoria, British Columbia, Canada",Private,?,?,Curtiss seaplane,?,?,1 (passengers:0 crew:1),1 (passengers:0 crew:1),0,The first fatal airplane accident in Canada oc...
4,"September 09, 1913",1830,Over the North Sea,Military - German Navy,?,?,Zeppelin L-1 (airship),?,?,20 (passengers:? crew:?),14 (passengers:? crew:?),0,The airship flew into a thunderstorm and encou...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5058,"July 16, 2022",2247,"Eleftheroupolis, Greece",Meridian,MEM3032,Nis- Amman,Antonov An-12,UR-CIC,01347701,8 (passengers:0 crew:8),8 (passengers:0 crew:8),0,The cargo plane carrying eight crew members an...
5059,"November 06, 2022",0853,"Bukoba, Tanzania",Precision Air,PW494,Dar es-Salaam -Bukoba,ATR 42-500,5H-PWF,819,43 (passengers:39 crew:39),19 (passengers:17 crew:2),0,"While on final approach to Bukoba Airport, the..."
5060,"November 18, 2022",1511,"Lima, Peru",LATAM,LA2213,Lima - Juliaca,Airbus 320-271N,CC-BHB,7864,108 (passengers:102 crew:6),0 (passengers:0 crew:0),2,The Airbus A320 collided with a fire truck whi...
5061,"November 21, 2022",1015,"Medellín, Colombia",AeroPaca SAS,?,Medellín - Pizarro,Piper PA-31-350 Navajo Chieftain,HK-5121,31-7652004,8 (passengers:6 crew:2),8 (passengers:6 crew:2),0,The plane was chartered to carry a team of six...


#### Saving the dataframe in an excel file

In [10]:
crashes.to_excel('Data\plane_crash_info_data.xlsx', index=False)